In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import find

# TF-IDF Similarity
*add description here*

In [10]:
df = pd.read_csv("data/skincare_products.csv")
df.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [11]:
# Preprocess the ingredients
def preprocess_ingredients(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and special characters (keeping spaces)
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Apply the preprocessing function to the 'Ingredient Description' column
df['Ingredients'] = df['Ingredients'].apply(preprocess_ingredients)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the 'Ingredient Description' column
tfidf_matrix = vectorizer.fit_transform(df['Ingredients'])

# Inspect the shape of the resulting TF-IDF matrix
print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

# Print the first few rows of the TF-IDF matrix
print(tfidf_matrix[])

# Optionally, you can inspect the feature names (terms)
print("Feature names (first 10):", vectorizer.get_feature_names_out()[:10])


TF-IDF Matrix Shape: (1472, 5185)
  (0, 2044)	0.0526409751371897
  (0, 1488)	0.08227694646267746
  (0, 692)	0.05729678829940637
  (0, 1171)	0.08950622059978404
  (0, 4306)	0.08714564451709136
  (0, 704)	0.07629197448112127
  (0, 1186)	0.07792117744806454
  (0, 2430)	0.11695041239953834
  (0, 2828)	0.06282841444311962
  (0, 2128)	0.0783853444741499
  (0, 2823)	0.06340816726540947
  (0, 3537)	0.07503645947227491
  (0, 4587)	0.05640326361529597
  (0, 1403)	0.141877310867892
  (0, 207)	0.036754609252126105
  (0, 1179)	0.05769012530990288
  (0, 3345)	0.11403064276514618
  (0, 1644)	0.10066548195600045
  (0, 383)	0.0783853444741499
  (0, 3405)	0.10146750882050443
  (0, 1465)	0.11269371117126495
  (0, 714)	0.13140417713156607
  (0, 5105)	0.03052142426702687
  (0, 3258)	0.11403064276514618
  (0, 4639)	0.10544955083511623
  :	:
  (4, 692)	0.035203459107530366
  (4, 4306)	0.05354275910066303
  (4, 4587)	0.03465447267020548
  (4, 207)	0.15807562614879045
  (4, 1179)	0.03544512751121358
  (4, 383)

In [ ]:
tfid

In [12]:
# Get the feature names (terms)
feature_names = vectorizer.get_feature_names_out()

# Find non-zero entries and their indices and values
row_indices, term_indices, scores = find(tfidf_matrix)

# Combine the rows, term indices, and scores into a list of tuples
entries = list(zip(row_indices, term_indices, scores))

# Sort entries by score in descending order
entries_sorted = sorted(entries, key=lambda x: x[2], reverse=True)

# Print the top 10 terms with the highest TF-IDF scores
print("\nTop 10 terms with the highest TF-IDF scores:")
for i in range(min(10, len(entries_sorted))):
    row_index, term_index, score = entries_sorted[i]
    term = feature_names[term_index]
    print(f"Document {row_index}: Term '{term}', Score: {score}")



Top 10 terms with the highest TF-IDF scores:
Document 39: Term 'name', Score: 1.0
Document 93: Term 'natural', Score: 1.0
Document 263: Term 'name', Score: 1.0
Document 310: Term 'hadasei3', Score: 1.0
Document 496: Term 'name', Score: 1.0
Document 656: Term 'name', Score: 1.0
Document 846: Term 'hadasei3', Score: 1.0
Document 90: Term 'essential', Score: 0.9330613898050689
Document 121: Term 'essential', Score: 0.9330613898050689
Document 858: Term 'essential', Score: 0.9330613898050689


In [13]:
# Compute the mean TF-IDF score for each term across the entire dataset
mean_tfidf_scores = tfidf_matrix.mean(axis=0).A1

# Get the feature names (terms)
terms = vectorizer.get_feature_names_out()

# Create a DataFrame to map terms to their mean TF-IDF scores
mean_tfidf_df = pd.DataFrame({'term': terms, 'mean_tfidf': mean_tfidf_scores})

# Sort the DataFrame by mean TF-IDF scores in descending order
mean_tfidf_df = mean_tfidf_df.sort_values(by='mean_tfidf', ascending=False)

# Display the top 10 terms with the highest mean TF-IDF scores
top_10_terms = mean_tfidf_df.head(10)
print("Top 10 terms with the highest mean TF-IDF scores across the entire dataset:")
print(top_10_terms)


Top 10 terms with the highest mean TF-IDF scores across the entire dataset:
         term  mean_tfidf
1878  extract    0.125674
3370      oil    0.070623
4489   sodium    0.063218
207      acid    0.053459
2201   glycol    0.052736
4376     seed    0.039070
2761     leaf    0.037237
2066    fruit    0.036676
1149       ci    0.035016
333   alcohol    0.034404


In [19]:
# Function to calculate similarity score between two products based on their ingredients
def calculate_similarity(product1_ingredients, product2_ingredients):
    # Preprocess ingredient descriptions
    product1_ingredients = preprocess_ingredients(product1_ingredients)
    product2_ingredients = preprocess_ingredients(product2_ingredients)
    
    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the ingredient descriptions of the two products
    tfidf_matrix_product1 = vectorizer.fit_transform([product1_ingredients])
    tfidf_matrix_product2 = vectorizer.transform([product2_ingredients])

    # Compute cosine similarity between the TF-IDF vectors of the two products
    similarity_score = cosine_similarity(tfidf_matrix_product1, tfidf_matrix_product2)

    return similarity_score[0][0]

# Example usage:
product1_ingredients = df.loc[1, 'Ingredients']
product2_ingredients = df.loc[2, 'Ingredients']
similarity_score = calculate_similarity(product1_ingredients, product2_ingredients)
print("Similarity Score between Product 1 and Product 2:", similarity_score)


Similarity Score between Product 1 and Product 2: 0.6071428571428571


In [18]:
df.loc[1,'Ingredients']

'galactomyces ferment filtrate pitera butylene glycol pentylene glycol water sodium benzoate methylparaben sorbic acid'

In [20]:

# Preprocess the ingredients
def preprocess_ingredients(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and special characters (keeping spaces)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove specified words
    words_to_remove = ['the', 'of', 'with', 'is']
    text = ' '.join(word for word in text.split() if word not in words_to_remove)
    return text

# Apply the preprocessing function to the 'Ingredient Description' column
df['Ingredients'] = df['Ingredients'].apply(preprocess_ingredients)

# Function to calculate similarity score between two products based on their ingredients
def calculate_similarity(product1_ingredients, product2_ingredients):
    # Preprocess ingredient descriptions
    product1_ingredients = preprocess_ingredients(product1_ingredients)
    product2_ingredients = preprocess_ingredients(product2_ingredients)
    
    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the ingredient descriptions of the two products
    tfidf_matrix_product1 = vectorizer.fit_transform([product1_ingredients])
    tfidf_matrix_product2 = vectorizer.transform([product2_ingredients])

    # Compute cosine similarity between the TF-IDF vectors of the two products
    similarity_score = cosine_similarity(tfidf_matrix_product1, tfidf_matrix_product2)

    return similarity_score[0][0]

# Function to calculate similarity scores between any two products
def calculate_similarity_scores(product1_index, product2_index):
    product1_ingredients = df.loc[product1_index, 'Ingredients']
    product2_ingredients = df.loc[product2_index, 'Ingredients']
    similarity_score = calculate_similarity(product1_ingredients, product2_ingredients)
    return similarity_score

# Example usage:
product1_index = int(input("Enter the index of the first product: "))
product2_index = int(input("Enter the index of the second product: "))
similarity_score = calculate_similarity_scores(product1_index, product2_index)
print(f"Similarity Score between Product {product1_index} and Product {product2_index}: {similarity_score}")


Similarity Score between Product 5 and Product 10: 0.5746645758132957
